# Task 3 for data science capstone project - Anthony

# 1. Copying the work done before in Task 2

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [4]:
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal Code,,Borough,,Neighbourhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


In [5]:
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postal Code,,Borough,,Neighbourhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


In [6]:
df5 = df4.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df5.reset_index(inplace = True)
df5.head()

,Postal Code,Borough,,,,Neighbourhood,
0,M3A,North York,,,,Parkwoods,
1,M4A,North York,,,,Victoria Village,
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",


In [7]:
df5.shape

(103, 7)

In [8]:
url = "http://cocl.us/Geospatial_data"
df6 = pd.read_csv(url)
df6.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
latandlong = pd.merge(df5, df6, on='Postal Code')
latandlong.head(12)

,Postal Code,Borough,,,,Neighbourhood,,Latitude,Longitude
0,M3A,North York,,,,Parkwoods,,43.753259,-79.329656
1,M4A,North York,,,,Victoria Village,,43.725882,-79.315572
2,M5A,Downtown Toronto,,,,"Regent Park, Harbourfront",,43.654260,-79.360636
3,M6A,North York,,,,"Lawrence Manor, Lawrence Heights",,43.718518,-79.464763
4,M7A,Downtown Toronto,,,,"Queen's Park, Ontario Provincial Government",,43.662301,-79.389494
5,M9A,Etobicoke,,,,"Islington Avenue, Humber Valley Village",,43.667856,-79.532242
6,M1B,Scarborough,,,,"Malvern, Rouge",,43.806686,-79.194353
7,M3B,North York,,,,Don Mills,,43.745906,-79.352188
8,M4B,East York,,,,"Parkview Hill, Woodbine Gardens",,43.706397,-79.309937
9,M5B,Downtown Toronto,,,,"Garden District, Ryerson",,43.657162,-79.378937


# 2. Seeing how many times "Central Toronto" has appeared

In [35]:
centraltoronto=latandlong[latandlong['Borough'].str.contains('Central')]
centraltoronto

,Postal Code,Borough,,,,Neighbourhood,,Latitude,Longitude
61,M4N,Central Toronto,,,,Lawrence Park,,43.728020,-79.388790
62,M5N,Central Toronto,,,,Roselawn,,43.711695,-79.416936
67,M4P,Central Toronto,,,,Davisville North,,43.712751,-79.390197
68,M5P,Central Toronto,,,,"Forest Hill North & West, Forest Hill Road Park",,43.696948,-79.411307
73,M4R,Central Toronto,,,,"North Toronto West, Lawrence Park",,43.715383,-79.405678
74,M5R,Central Toronto,,,,"The Annex, North Midtown, Yorkville",,43.672710,-79.405678
79,M4S,Central Toronto,,,,Davisville,,43.704324,-79.388790
83,M4T,Central Toronto,,,,"Moore Park, Summerhill East",,43.689574,-79.383160
86,M4V,Central Toronto,,,,"Summerhill West, Rathnelly, South Hill, Forest...",,43.686412,-79.400049


# 3. Generating the map to visualize the clustering

# 3.1 Installing geopy to be able to use Nominatim

In [32]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\acesa\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0




geopy-2.1.0          | 64 KB     |            |   0% 
geopy-2.1.0          | 64 KB     | ##5        |  25% 
geopy-2.1.0          | 64 KB     | ########## | 100% 
geopy-2.1.0          | 64 KB     | ########## | 100% 

geographiclib-1.50  

# 3.2 Map visualization

In [45]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

centraltoronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(centraltoronto['Latitude'], centraltoronto['Longitude'], 
                                           centraltoronto['Borough'], centraltoronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(centraltoronto_map)  
    
centraltoronto_map